In [1]:
import paramiko

ip = "172.16.16.25"
user = "root"
pw = "asd123!@"

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err


def mariaDB_install():
    cmd =  """
    sudo dnf install -y expect
    dnf -y install dnf-plugins-core epel-release expect
    
    dnf install -y https://rpms.remirepo.net/enterprise/remi-release-9.rpm
    dnf module reset php -y
    dnf module enable php:remi-8.4 -y
    """
    print(cc(cmd))
def apache_php_install():
    #  Apache + PHP 설치
    cmd1 = r"""
    bash -c 'dnf -y install httpd php php-fpm php-mysqlnd php-cli php-common php-devel \
        php-mbstring php-xml php-gd php-json php-intl php-zip php-opcache'
    """
    cc(cmd1)

    print(cc("systemctl enable --now httpd php-fpm"))
    print(cc("systemctl status httpd php-fpm"))
def install_mariaDB() :

    cmd = """
    dnf install -y mariadb-server
    systemctl enable --now mariadb
    """
    print(cc("systemctl status mariadb"))
    
def init_mariaDB() :
    print("secure_installation 시작")
    MYSQL_ROOT_PASSWORD="asd123!@"
    cmd = f'''
cat <<EOF > /root/mysql_secure.exp
#!/usr/bin/expect -f
set timeout -1
set password [lindex $argv 0]

spawn mysql_secure_installation

expect "Enter current password for root (enter for none):"
send "\\r"
expect "Switch to unix_socket authentication"
send "Y\\r"
expect "Change the root password?"
send "y\\r"
expect "New password:"
send "{MYSQL_ROOT_PASSWORD}\\r"
expect "Re-enter new password:"
send "{MYSQL_ROOT_PASSWORD}\\r"
expect "Remove anonymous users?"
send "Y\\r"
expect "Disallow root login remotely?"
send "Y\\r"
expect "Remove test database and access to it?"
send "Y\\r"
expect "Reload privilege tables now?"
send "Y\\r"
expect eof
EOF

chmod +x /root/mysql_secure.exp
/usr/bin/expect /root/mysql_secure.exp "{MYSQL_ROOT_PASSWORD}"
'''
    
    print(cc(cmd))

def maria_in() :
#    mariaDB_install()
#    apache_php_install()
#    install_mariaDB()
    init_mariaDB()
maria_in()

secure_installation 시작
spawn mysql_secure_installation

NOTE: RUNNING ALL PARTS OF THIS SCRIPT IS RECOMMENDED FOR ALL MariaDB
      SERVERS IN PRODUCTION USE!  PLEASE READ EACH STEP CAREFULLY!

In order to log into MariaDB to secure it, we'll need the current
password for the root user. If you've just installed MariaDB, and
haven't set the root password yet, you should just press enter here.

Enter current password for root (enter for none): 
OK, successfully used password, moving on...

Setting the root password or using the unix_socket ensures that nobody
can log into the MariaDB root user without the proper authorisation.

You already have your root account protected, so you can safely answer 'n'.

Switch to unix_socket authentication [Y/n] Y
Enabled successfully!
Reloading privilege tables..
 ... Success!


You already have your root account protected, so you can safely answer 'n'.

Change the root password? [Y/n] y
New password: 
Re-enter new password: 
Password updated successful